# Workflow

In [1]:
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
import sys

sys.path.append('../hydra_tod/')

from astropy.coordinates import EarthLocation, AltAz, SkyCoord
from astropy.time import Time, TimeDelta
import astropy.units as u
from utils import Leg_poly_proj, view_samples
from flicker_model import sim_noise, flicker_cov
from simulation import * 

In [2]:
# Load the simulation results
import pickle
from simulation import TODSimulation

with open('tod_simulation_single.pkl', 'rb') as f:
    tod_sim = pickle.load(f)

In [3]:
sky_params = tod_sim.sky_params
nd_rec_params = tod_sim.nd_rec_params
gain_params_setting = tod_sim.gain_params_setting
calibration_indices = tod_sim.calibration_1_indice
alpha = tod_sim.alpha

TOD_setting = tod_sim.TOD_setting
t_list = tod_sim.t_list
gain_proj = tod_sim.gain_proj
Tsky_operator_setting = tod_sim.Tsky_operator_setting
nd_rec_operator = tod_sim.nd_rec_operator

logfc = tod_sim.logfc

# Additional variables needed for the analysis
pixel_indices = tod_sim.pixel_indices
nside = tod_sim.nside
f0 = tod_sim.f0
T_ndiode = tod_sim.T_ndiode
rec_params = tod_sim.rec_params
logf0_list = [tod_sim.logf0]  # Convert to list format as expected later



In [4]:
init_Tsys_params = np.hstack([sky_params, nd_rec_params])
init_noise_params = [-4.8, 2.2]

gain_prior_cov_inv = np.zeros(4)
gain_prior_cov_inv[0] = 2.51 # DC mode: Prior STD ~10%
gain_prior_mean = gain_params_setting

prior_std_frac = 0.1
Tsky_prior_cov_inv = 1. / (prior_std_frac * sky_params)**2
Tsky_prior_cov_inv[calibration_indices] = 1e20

aux = np.ones_like(nd_rec_params)*0.0
aux[0] = 1
Trec_prior_cov_inv = [aux] 


def log_prior_noise(params):
    _, alpha_est = params
    return -1e10 * (alpha_est - alpha)**4 


In [5]:
# Import the FlickerCorrEmulator class first
from flicker_model import FlickerCorrEmulator
from full_Gibbs_sampler import full_Gibbs_sampler_multi_TODS_v2

Tsys_samples, all_gain_samples, all_noise_samples = \
    full_Gibbs_sampler_multi_TODS_v2([TOD_setting],
                                    [t_list],
                                    [gain_proj],
                                    [Tsky_operator_setting],
                                    [nd_rec_operator],
                                    init_Tsys_params,
                                    init_noise_params, 
                                    [logfc],
                                    num_Jeffrey=False,
                                    wnoise_var=2.5e-6,
                                    Tsky_prior_cov_inv=Tsky_prior_cov_inv,
                                    Tsky_prior_mean=sky_params,
                                    local_Trec_prior_cov_inv_list=Trec_prior_cov_inv,
                                    local_Trec_prior_mean_list=[nd_rec_params],
                                    local_gain_prior_cov_inv_list=[gain_prior_cov_inv],
                                    local_gain_prior_mean_list=[gain_prior_mean],
                                    local_noise_prior_func_list=[log_prior_noise],
                                    n_samples=1000,
                                    tol=1e-12)

# First, save the samples as npy files..

np.save("outputs/GSF5/Tsys_samples.npy", Tsys_samples)
np.save("outputs/GSF5/gain_samples.npy", np.concatenate(all_gain_samples, axis=0))
np.save("outputs/GSF5/noise_samples.npy", np.concatenate(all_noise_samples, axis=0))

Using the emulator for flicker noise correlation function.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 0: [6.31221083 0.41911791 0.26178674 0.05663032]
Rank: 0, local id: 0, noise_sample 0: [-4.71251783  2.00115341]
Reached max iterations with fractional norm error 0.0004366597567246937.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 1: [6.31295041 0.42084853 0.26282374 0.0585093 ]
Rank: 0, local id: 0, noise_sample 1: [-5.16405167  1.99929276]
Reached max iterations with fractional norm error 0.000628210180321211.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 2: [6.31331103 0.42090523 0.26517301 0.0580582 ]
Rank: 0, local id: 0, noise_sample 2: [-4.55497694  2.01578609]
Reached max iterations with fractional norm error 5.659467765387444e-05.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 3: [6.31353078 0.42289755 0.26725748 0.06004291]
Rank: 0, local id: 0, noise_sample 3: [-3.57323501  2.00376118]
Reached max iterations with